Recall this code from the last exercise:

In [51]:
import random
from datetime import datetime

class Table():
    def __init__(self):
        self._data = {}
        self.validation = lambda x: True
        
    def get(self, key):
        if key not in self._data.keys():
            return None
        
        return self._data[key]
    
    def set(self, key, value, *validation_args):
        self._data[key] = value

We use **transactions** in databases to wrap operations and make sure everything is going to proceed properly before we commit the change to the database.

In [53]:
class NoTransactionInProgressException(Exception):
    def __str__(self):
        return "There isn't a transaction in progress!"
    
class TableAlreadyExistsException(Exception):
    def __str__(self):
        return "A table by that name already exists!"

class Database():
    def __init__(self):
        self.tables = {}
        self.transaction_in_progress = False
        
    def create_table(self, name):
        if name in self.tables.keys():
            raise TableAlreadyExistsException()
        
        self.tables[name] = Table()
        setattr(self, name, self.tables[name])
                
    def begin(self):
        for table in self.tables:
            self.tables[table]._before_transaction = self.tables[table]._data.copy()
        self.transaction_in_progress = True
    
    def commit(self):
        if not self.transaction_in_progress:
            raise NoTransactionInProgressException()
        for table in self.tables:
            self.tables[table]._before_transaction = {}
        self.transaction_in_progress = False
       
    def rollback(self):
        if not self.transaction_in_progress:
            raise NoTransactionInProgressException()
        for table in self.tables:
            self.tables[table]._data = self.tables[table]._before_transaction
            self.tables[table]._before_transaction = {}
        self.transaction_in_progress = False

Here are some tests that demonstrate the transactions at work.

In [54]:
import sys
!{sys.executable} -m pip install colorama 

sys.path.insert(0, '..')
from test_framework_exercise.phoenix_test.matchers import FailedAssertion, Assertion, assert_that
from test_framework_exercise.phoenix_test.test import Test
sys.path.remove('..')

You should consider upgrading via the '/Users/chelseatroy/.pyenv/versions/3.9.0/bin/python3.9 -m pip install --upgrade pip' command.


## Committing

In [55]:
class TransactionAndCommitTest(Test):
    def test_begin_commit(self):
        db = Database()
        db.create_table("birds_on_youtube")
        
        db.begin()
        db.birds_on_youtube.set("Vinny", {"type": "Galah", "owner": "Marlene McCohen", "interests" : "music"})
        db.commit()
        
        assert_that(db.birds_on_youtube.get("Vinny")).equals({"type": "Galah", "owner": "Marlene McCohen", "interests" : "music"})
        
    def test_commit_without_transaction(self):
        db = Database()
        
        try:
            db.commit()
            assert_that("No Exception Thrown").equals("NoTransactionInProgressException")
        except NoTransactionInProgressException:
            return
        
        assert_that("Wrong Exception Thrown").equals("NoTransactionInProgressException")

TransactionAndCommitTest().run()

    test_begin_commit passed!
    test_commit_without_transaction passed!
    2 out of 2 tests passed.



(2, 2)

## Rolling Back

In [56]:
class TransactionAndRollbackTest(Test):
    def test_rollback_with_transaction(self):
        db = Database()
        db.create_table("birds_on_youtube")
        
        db.begin()
        db.birds_on_youtube.set("Vinny", {"type": "Galah", "owner": "Marlene McCohen", "interests" : "music"})
        db.rollback()

        assert_that(db.birds_on_youtube.get("Vinny")).equals(None)

    def test_rollback_no_transaction(self):
        db = Database()
        
        try:
            db.rollback()
            assert_that("No Exception Thrown").equals("NoTransactionInProgressException")
        except NoTransactionInProgressException:
            return
        
        assert_that("Wrong Exception Thrown").equals("NoTransactionInProgressException")

TransactionAndRollbackTest().run()

    test_rollback_no_transaction passed!
    test_rollback_with_transaction passed!
    2 out of 2 tests passed.



(2, 2)

## Here's an example of how a transaction might be useful.

Suppose you run a pet shop, and you want local bird clubs to be able to enter their members' menageries into your system to get customized emails about discounts especially for their pets. For example "Try out this new birdie boom box! With easy, push-button access to three great tunes. Vinny will love it!" (Sidenote: Vinny actually would love this. However, **DO NOT** give a bird control over a noise-making device if you ever want to sleep again). 

What you DON'T want in your system is a bunch of birds without a recorded owner, because that's just taking up space in your system without giving you a contact person to sell stuff to. So you want to reject uploads from local bird clubs where ANY of the birds in the upload don't have an owner. That said, it's easier for the bird clubs to fix their files and re-upload then go back, take out just the invalid rows, and ONLY upload those. So, you reject the whole upload if there's even one problem.

Try changing one of the `owner_name`s in `birds_to_add` to a name that does not appear in the `bird_owners` table, and see what gets inserted into `birds_to_add`.

In [57]:
db = Database()

db.create_table('bird_owners')
db.bird_owners.set("Marlene", {"city": "Chatsworth", "email": "mmccohen@sniffers.com"})
db.bird_owners.set("Sister", {"city": "Chatsworth", "email": "mm1234@birdmail.com"})

db.create_table('birds_on_youtube')
birds_to_add = {
    "Vinny": {"type": "Galah", "owner_name": "Marlene", "interests": "music"},
    "Picasso": {"type": "Mustache Parakeet", "owner_name": "Marlene", "interests": "fruits"},
    "Jersey": {"type": "Umbrella Cockatoo", "owner_name": "Marlene", "interests": "screaming"},
    "George": {"type": "African Grey", "owner_name": "Marlene", "interests": "books"},
    "Finchey": {"type": "Finch", "owner_name": "Marlene", "interests": "seed"},
    "Rocky": {"type": "Catalina Maccaw", "owner_name": "Marlene", "interests": "dancing"},
    "Leo": {"type": "Red-Lored Amazon", "owner_name": "Marlene", "interests": "Rocky"},
    "Blu": {"type": "Indian Ringneck", "owner_name": "Sister", "interests": "Leo"},
}

db.begin()
for bird_name, bird in birds_to_add.items():
    if birds_to_add[bird_name]['owner_name'] in db.bird_owners._data.keys():
        db.birds_on_youtube.set(bird_name, bird)
    else:
        db.rollback()
        break

print(db.birds_on_youtube._data)

{'Vinny': {'type': 'Galah', 'owner_name': 'Marlene', 'interests': 'music'}, 'Picasso': {'type': 'Mustache Parakeet', 'owner_name': 'Marlene', 'interests': 'fruits'}, 'Jersey': {'type': 'Umbrella Cockatoo', 'owner_name': 'Marlene', 'interests': 'screaming'}, 'George': {'type': 'African Grey', 'owner_name': 'Marlene', 'interests': 'books'}, 'Finchey': {'type': 'Finch', 'owner_name': 'Marlene', 'interests': 'seed'}, 'Rocky': {'type': 'Catalina Maccaw', 'owner_name': 'Marlene', 'interests': 'dancing'}, 'Leo': {'type': 'Red-Lored Amazon', 'owner_name': 'Marlene', 'interests': 'Rocky'}, 'Blu': {'type': 'Indian Ringneck', 'owner_name': 'Sister', 'interests': 'Leo'}}


Now put it back so that the transaction goes through, because we need some stuff in the `birds_on_youtube` table for this next part.

Let's calculate how much space the database takes up during a transaction:

In [58]:
import sys

def get_bytes(table):
    return sys.getsizeof(str(table._data)) + sys.getsizeof(str(table._before_transaction))

def calculate_table_space_for(database):
    return sum([get_bytes(table) for table in database.tables.values()])

In [59]:
db.begin()
calculate_table_space_for(db)

1800

In [60]:
db.rollback()
calculate_table_space_for(db)

1002

Neither of these numbers is a lot of bytes, but they demonstrate just how space-consuming our current implementation of transactions is. It almost doubles the amount of space we need to store data during a transaction?

### Challenge: Can you find a way to make transactions work without requiring so much space?